In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11775377860863669033
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 10596170577812394798
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# declare ImageDataGenerator and flow_from_directory vars
batch_size=10
num_classes = 120

In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
#obtain train and test labels
from scipy.io import loadmat

#y_train_rough = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']

#y_test = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_list.mat''')['labels']

In [6]:
from sklearn.model_selection import train_test_split

In [21]:
labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']
files = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['file_list']
labels = [item for label in labels for item in label] #this is flattening a list of lists, because for some reason ever label is stored as a list
files = [item for file in files for item in file]
df = pd.DataFrame({'labels':labels, 'files':files})
train, validate = train_test_split(df, test_size = 0.2, stratify=labels)


In [22]:
X_train = train['files']
y_train = to_categorical(train['labels'])
X_val = validate['files']
y_val = to_categorical(validate['labels'])

In [57]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=10)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=10)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=10)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [24]:
#obtain validation labels
# no longer used, utilizing sklearn's train_test_split function

'''
y_validation = []
y_train = []

for i in range(120):
    i = i*100
    
    #begin_index = train_full_list[i::100]
    #end_index = train_full_list[i+100::100]
    
    slice = y_train_rough[i:i+100,]
    y_train.append(slice[:80])
    y_validation.append(slice[80:])

y_train = np.concatenate(y_train, axis=0)
y_validation = np.concatenate(y_validation, axis=0)
'''

'\ny_validation = []\ny_train = []\n\nfor i in range(120):\n    i = i*100\n    \n    #begin_index = train_full_list[i::100]\n    #end_index = train_full_list[i+100::100]\n    \n    slice = y_train_rough[i:i+100,]\n    y_train.append(slice[:80])\n    y_validation.append(slice[80:])\n\ny_train = np.concatenate(y_train, axis=0)\ny_validation = np.concatenate(y_validation, axis=0)\n'

In [6]:
# using a simple CNN to start

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.layers.normalization import BatchNormalization

import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [7]:
input_shape = (224,224, 3)

# create the model

base_model = Sequential()
base_model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model.add(Flatten())

base_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model.add(Dropout(0.2))
base_model.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
__________

In [13]:
# train/fit the simple CNN using flow from directory 

base_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=25, callbacks=[early_stopping])

Epoch 1/25
300/300 [==============================] - 158s 526ms/step - loss: 4.7910 - acc: 0.0075 - val_loss: 4.7881 - val_acc: 0.0079
Epoch 2/25
300/300 [==============================] - 156s 519ms/step - loss: 4.7896 - acc: 0.0088 - val_loss: 4.7874 - val_acc: 0.0096
Epoch 3/25
300/300 [==============================] - 152s 507ms/step - loss: 4.7915 - acc: 0.0088 - val_loss: 4.7869 - val_acc: 0.0083
Epoch 4/25
300/300 [==============================] - 153s 508ms/step - loss: 4.7880 - acc: 0.0088 - val_loss: 4.7870 - val_acc: 0.0083
Epoch 5/25
300/300 [==============================] - 154s 513ms/step - loss: 4.7886 - acc: 0.0083 - val_loss: 4.7866 - val_acc: 0.0071
Epoch 6/25
300/300 [==============================] - 158s 526ms/step - loss: 4.7872 - acc: 0.0089 - val_loss: 4.7862 - val_acc: 0.0096
Epoch 7/25
300/300 [==============================] - 155s 516ms/step - loss: 4.7890 - acc: 0.0092 - val_loss: 4.7861 - val_acc: 0.0096
Epoch 8/25
300/300 [============================

In [14]:
base_model.save('base_model.h5')

In [13]:
# try base_model with Batch normalization

base_model_BN = Sequential()
base_model_BN.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model_BN.add(BatchNormalization())
base_model_BN.add(MaxPooling2D(pool_size=(2, 2)))


base_model_BN.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_BN.add(MaxPooling2D(pool_size=(2, 2)))
base_model_BN.add(BatchNormalization())

base_model_BN.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_BN.add(Flatten())

base_model_BN.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model_BN.add(Dropout(0.2))
base_model_BN.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model_BN.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model_BN.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 54, 54, 64)        23296     
_________________________________________________________________
batch_normalization_11 (Batc (None, 54, 54, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 2, 2, 64)          65600     
__________

In [14]:
base_model_BN.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=10, callbacks=[early_stopping])

Epoch 1/10
300/300 [==============================] - 241s 804ms/step - loss: 4.9950 - acc: 0.0068 - val_loss: 4.9415 - val_acc: 0.0104
Epoch 2/10
300/300 [==============================] - 166s 552ms/step - loss: 4.9400 - acc: 0.0089 - val_loss: 4.9159 - val_acc: 0.0112
Epoch 3/10
300/300 [==============================] - 157s 522ms/step - loss: 4.9147 - acc: 0.0101 - val_loss: 4.8875 - val_acc: 0.0117
Epoch 4/10
300/300 [==============================] - 158s 527ms/step - loss: 4.8823 - acc: 0.0089 - val_loss: 4.8713 - val_acc: 0.0108
Epoch 5/10
300/300 [==============================] - 165s 551ms/step - loss: 4.8709 - acc: 0.0081 - val_loss: 4.8565 - val_acc: 0.0117
Epoch 6/10
300/300 [==============================] - 161s 536ms/step - loss: 4.8615 - acc: 0.0100 - val_loss: 4.8484 - val_acc: 0.0108
Epoch 7/10
300/300 [==============================] - 156s 520ms/step - loss: 4.8662 - acc: 0.0099 - val_loss: 4.8323 - val_acc: 0.0096
Epoch 8/10
300/300 [============================

In [15]:
base_model_BN.save('base_model_BN.h5')

In [20]:
# tweak base model... decrease first covnet filter, decease filter size and stride, add Batch Normalization

base_model_BN_v2 = Sequential()
base_model_BN_v2.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model_BN_v2.add(BatchNormalization())
base_model_BN_v2.add(MaxPooling2D(pool_size=(2, 2)))
base_model_BN_v2.add(Dropout(0.2))

base_model_BN_v2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_BN_v2.add(BatchNormalization())
base_model_BN_v2.add(MaxPooling2D(pool_size=(2, 2)))
base_model_BN_v2.add(Dropout(0.2))

base_model_BN_v2.add(Conv2D(32, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_BN_v2.add(BatchNormalization())
base_model_BN_v2.add(MaxPooling2D(pool_size=(2, 2)))
base_model_BN_v2.add(Dropout(0.2))
base_model_BN_v2.add(Flatten())

base_model_BN_v2.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
base_model_BN_v2.add(BatchNormalization())
base_model_BN_v2.add(Dropout(0.2))

base_model_BN_v2.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model_BN_v2.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model_BN_v2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
batch_normalization_29 (Batc (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
batch_normalization_30 (Batc (None, 109, 109, 32)      128       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 54, 54, 32)        0         
__________

In [21]:
base_model_BN_v2.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=10, callbacks=[early_stopping])

Epoch 1/10
300/300 [==============================] - 166s 552ms/step - loss: 5.5849 - acc: 0.0077 - val_loss: 5.1300 - val_acc: 0.0100
Epoch 2/10
300/300 [==============================] - 161s 535ms/step - loss: 5.5356 - acc: 0.0077 - val_loss: 5.1100 - val_acc: 0.0104
Epoch 3/10
300/300 [==============================] - 162s 539ms/step - loss: 5.4870 - acc: 0.0109 - val_loss: 5.0985 - val_acc: 0.0100
Epoch 4/10
300/300 [==============================] - 169s 562ms/step - loss: 5.4590 - acc: 0.0083 - val_loss: 5.0972 - val_acc: 0.0112
Epoch 5/10
300/300 [==============================] - 166s 554ms/step - loss: 5.4164 - acc: 0.0089 - val_loss: 5.0881 - val_acc: 0.0092
Epoch 6/10
300/300 [==============================] - 177s 588ms/step - loss: 5.3747 - acc: 0.0081 - val_loss: 5.1088 - val_acc: 0.0092
Epoch 7/10
 70/300 [======>.......................] - ETA: 2:18 - loss: 5.3490 - acc: 0.0097

KeyboardInterrupt: 

In [ ]:
# trying a larger FC layer

base_model_v3 = Sequential()
base_model_v3.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
base_model_v3.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v3.add(Conv2D(32, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v3.add(MaxPooling2D(pool_size=(2, 2)))

base_model_v3.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
base_model_v3.add(Flatten())

base_model_v3.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
base_model_v3.add(Dropout(0.2))
base_model_v3.add(Dense(num_classes, activation='softmax'))

# Compile model
adam_op = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
base_model_v3.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(base_model_v3.summary())

In [11]:
# trying deep_model architecture (in this case, adding additional conv layer and pooling after 2 conv +activation layers

deep_model = Sequential()

deep_model = Sequential()
deep_model.add(Conv2D(96, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model.add(MaxPooling2D(pool_size=(2, 2)))

deep_model.add(Flatten())

deep_model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
deep_model.add(Dropout(0.2))
deep_model.add(Dense(num_classes, activation='softmax'))
    
# Compile model
# use existing adam optimizer
deep_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 25, 25, 64)        55360     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 23, 23, 64)        36928     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 9, 9, 32)          18464     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 7, 7, 32)          9248      
__________

In [12]:
deep_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=300, epochs=50, callbacks=[early_stopping])

# not enough steps per epoch

Epoch 1/50
300/300 [==============================] - 216s 720ms/step - loss: 4.7901 - acc: 0.0076 - val_loss: 4.7877 - val_acc: 0.0071
Epoch 2/50
300/300 [==============================] - 156s 520ms/step - loss: 4.7890 - acc: 0.0063 - val_loss: 4.7873 - val_acc: 0.0063
Epoch 3/50
300/300 [==============================] - 150s 499ms/step - loss: 4.7874 - acc: 0.0072 - val_loss: 4.7871 - val_acc: 0.0121
Epoch 4/50
300/300 [==============================] - 155s 517ms/step - loss: 4.7895 - acc: 0.0064 - val_loss: 4.7872 - val_acc: 0.0071
Epoch 5/50
300/300 [==============================] - 156s 520ms/step - loss: 4.7877 - acc: 0.0085 - val_loss: 4.7871 - val_acc: 0.0079
Epoch 6/50
300/300 [==============================] - 153s 509ms/step - loss: 4.7875 - acc: 0.0091 - val_loss: 4.7870 - val_acc: 0.0088
Epoch 7/50
300/300 [==============================] - 156s 519ms/step - loss: 4.7879 - acc: 0.0087 - val_loss: 4.7868 - val_acc: 0.0092
Epoch 8/50
300/300 [============================

In [14]:
deep_model.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=25, callbacks=[early_stopping])

Epoch 1/25
800/800 [==============================] - 410s 512ms/step - loss: 4.7871 - acc: 0.0096 - val_loss: 4.7863 - val_acc: 0.0100
Epoch 2/25
800/800 [==============================] - 422s 527ms/step - loss: 4.7871 - acc: 0.0090 - val_loss: 4.7855 - val_acc: 0.0125
Epoch 3/25
800/800 [==============================] - 419s 524ms/step - loss: 4.7865 - acc: 0.0082 - val_loss: 4.7846 - val_acc: 0.0108
Epoch 4/25
800/800 [==============================] - 407s 509ms/step - loss: 4.7850 - acc: 0.0117 - val_loss: 4.7827 - val_acc: 0.0087
Epoch 5/25
800/800 [==============================] - 405s 506ms/step - loss: 4.7836 - acc: 0.0097 - val_loss: 4.7805 - val_acc: 0.0112
Epoch 6/25
800/800 [==============================] - 403s 504ms/step - loss: 4.7829 - acc: 0.0099 - val_loss: 4.7776 - val_acc: 0.0100
Epoch 7/25
800/800 [==============================] - 412s 515ms/step - loss: 4.7767 - acc: 0.0127 - val_loss: 4.7714 - val_acc: 0.0104
Epoch 8/25
800/800 [============================

In [15]:
deep_model.save('deep_model.h5')

In [29]:
# trying deep model with batch normalization and bigger FC layer
deep_model_BN = Sequential()
deep_model_BN.add(Conv2D(96, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
deep_model_BN.add(MaxPooling2D(pool_size=(2, 2)))
deep_model_BN.add(BatchNormalization())

deep_model_BN.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_BN.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_BN.add(MaxPooling2D(pool_size=(2, 2)))
deep_model_BN.add(BatchNormalization())

deep_model_BN.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_BN.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
deep_model_BN.add(MaxPooling2D(pool_size=(2, 2)))
deep_model_BN.add(BatchNormalization())
deep_model_BN.add(Flatten())

deep_model_BN.add(Dense(1028, activation='relu', kernel_constraint=maxnorm(3)))
deep_model_BN.add(Dropout(0.2))
deep_model_BN.add(Dense(num_classes, activation='softmax'))
    
# Compile model
# use existing adam optimizer
deep_model_BN.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(deep_model_BN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_54 (Batc (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 25, 25, 64)        55360     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 23, 23, 64)        36928     
_________________________________________________________________
batch_normalization_55 (Batc (None, 23, 23, 64)        256       
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 11, 11, 64)        0         
__________

In [30]:
deep_model_BN.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=10, callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 424s 530ms/step - loss: 5.1990 - acc: 0.0086 - val_loss: 4.9286 - val_acc: 0.0133
Epoch 2/10
800/800 [==============================] - 419s 524ms/step - loss: 4.9617 - acc: 0.0112 - val_loss: 4.8573 - val_acc: 0.0137
Epoch 3/10
800/800 [==============================] - 410s 513ms/step - loss: 4.8990 - acc: 0.0109 - val_loss: 4.8128 - val_acc: 0.0150
Epoch 4/10
800/800 [==============================] - 424s 529ms/step - loss: 4.8545 - acc: 0.0117 - val_loss: 4.7910 - val_acc: 0.0142
Epoch 5/10
800/800 [==============================] - 437s 547ms/step - loss: 4.8269 - acc: 0.0142 - val_loss: 4.7690 - val_acc: 0.0137
Epoch 6/10
800/800 [==============================] - 400s 500ms/step - loss: 4.8040 - acc: 0.0145 - val_loss: 4.7478 - val_acc: 0.0200
Epoch 7/10
800/800 [==============================] - 412s 515ms/step - loss: 4.7774 - acc: 0.0149 - val_loss: 4.7314 - val_acc: 0.0137
Epoch 8/10
800/800 [============================

In [31]:
deep_model_BN.save('deep_model_BN.h5')

In [58]:
# old iteration, retry with Batch Normalization and Global Average Pooling
# note to self, may try using this on pretrained VGG16 model to reduce size of FC layer

from keras.layers import GlobalAveragePooling2D

model_3 = Sequential()
model_3.add(BatchNormalization(input_shape=input_shape))
model_3.add(Conv2D(32, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(BatchNormalization())
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))
model_3.add(MaxPooling2D(pool_size=(2, 2)))


model_3.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.2))
model_3.add(GlobalAveragePooling2D())

model_3.add(Dense(num_classes, activation='softmax'))
     

# Compile model
model_3.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(model_3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_79 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 222, 222, 32)      896       
_________________________________________________________________
batch_normalization_80 (Batc (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d_101 (MaxPoolin (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 109, 109, 32)      9248      
_________________________________________________________________
dropout_99 (Dropout)         (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_102 (MaxPoolin (None, 54, 54, 32)        0         
__________

In [59]:
model_3.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=300, epochs=10, callbacks=[early_stopping])

Epoch 1/10
300/300 [==============================] - 77s 255ms/step - loss: 4.8914 - acc: 0.0063 - val_loss: 4.8358 - val_acc: 0.0100
Epoch 2/10
300/300 [==============================] - 74s 246ms/step - loss: 4.8963 - acc: 0.0047 - val_loss: 4.8213 - val_acc: 0.0113
Epoch 3/10
300/300 [==============================] - 76s 254ms/step - loss: 4.8703 - acc: 0.0083 - val_loss: 4.8141 - val_acc: 0.0088
Epoch 4/10
300/300 [==============================] - 77s 257ms/step - loss: 4.8410 - acc: 0.0040 - val_loss: 4.8100 - val_acc: 0.0088
Epoch 5/10
300/300 [==============================] - 74s 246ms/step - loss: 4.8457 - acc: 0.0067 - val_loss: 4.8071 - val_acc: 0.0083
Epoch 6/10
300/300 [==============================] - 75s 250ms/step - loss: 4.8144 - acc: 0.0133 - val_loss: 4.8057 - val_acc: 0.0096
Epoch 7/10
300/300 [==============================] - 78s 259ms/step - loss: 4.8048 - acc: 0.0073 - val_loss: 4.8039 - val_acc: 0.0075
Epoch 8/10
300/300 [==============================] - 7

In [62]:
# DONT USE

incr_layer_model = Sequential()
incr_layer_model.add(BatchNormalization(input_shape=input_shape))
incr_layer_model.add(Conv2D(16, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
incr_layer_model.add(MaxPooling2D(pool_size=(2, 2)))

incr_layer_model.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
incr_layer_model.add(Dropout(0.2))
incr_layer_model.add(MaxPooling2D(pool_size=(2, 2)))

incr_layer_model.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
incr_layer_model.add(Dropout(0.2))
incr_layer_model.add(MaxPooling2D(pool_size=(2, 2)))

incr_layer_model.add(Conv2D(128, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
incr_layer_model.add(Dropout(0.2))
incr_layer_model.add(MaxPooling2D(pool_size=(2, 2)))

incr_layer_model.add(Conv2D(256, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
incr_layer_model.add(BatchNormalization())
incr_layer_model.add(Dropout(0.2))
incr_layer_model.add(MaxPooling2D(pool_size=(2, 2)))


incr_layer_model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
incr_layer_model.add(Dropout(0.2))
incr_layer_model.add(GlobalAveragePooling2D())

incr_layer_model.add(Dense(num_classes, activation='softmax'))
     

# Compile model
incr_layer_model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(incr_layer_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_88 (Batc (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_114 (MaxPoolin (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_136 (Conv2D)          (None, 109, 109, 32)      4640      
_________________________________________________________________
dropout_112 (Dropout)        (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_115 (MaxPoolin (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_137 (Conv2D)          (None, 52, 52, 64)        18496     
__________

In [94]:
wide_model_slow_learn = Sequential()

wide_model_slow_learn.add(BatchNormalization(input_shape=input_shape,))
wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))


wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_slow_learn.add(Dense(2048, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_slow_learn.add(Dropout(0.2))
wide_model_slow_learn.add(GlobalAveragePooling2D())

wide_model_slow_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_slow_learn.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model_slow_learn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_131 (Bat (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_256 (Conv2D)          (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_226 (MaxPoolin (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_257 (Conv2D)          (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_227 (MaxPoolin (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_258 (Conv2D)          (None, 52, 52, 64)        36928     
_________________________________________________________________
max_pooling2d_228 (MaxPoolin (None, 26, 26, 64)        0         
__________

In [96]:
wide_model_slow_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=10, callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 195s 244ms/step - loss: 4.7125 - acc: 0.0151 - val_loss: 4.7043 - val_acc: 0.0171
Epoch 2/10
800/800 [==============================] - 218s 273ms/step - loss: 4.6934 - acc: 0.0158 - val_loss: 4.6914 - val_acc: 0.0204
Epoch 3/10
800/800 [==============================] - 255s 318ms/step - loss: 4.6904 - acc: 0.0165 - val_loss: 4.6830 - val_acc: 0.0192
Epoch 4/10
800/800 [==============================] - 202s 253ms/step - loss: 4.6855 - acc: 0.0161 - val_loss: 4.6787 - val_acc: 0.0171
Epoch 5/10
800/800 [==============================] - 193s 241ms/step - loss: 4.6642 - acc: 0.0163 - val_loss: 4.6624 - val_acc: 0.0225
Epoch 6/10
800/800 [==============================] - 192s 240ms/step - loss: 4.6658 - acc: 0.0166 - val_loss: 4.6538 - val_acc: 0.0208
Epoch 7/10
800/800 [==============================] - 191s 239ms/step - loss: 4.6542 - acc: 0.0169 - val_loss: 4.6458 - val_acc: 0.0229
Epoch 8/10
800/800 [============================

In [97]:
wide_model_slow_learn.save('wide_model_slow_learn.h5')

In [82]:
wide_model_fast_learn = Sequential()
wide_model_fast_learn.add(Conv2D(64, (3, 3), strides=1, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_fast_learn.add(BatchNormalization())

wide_model_fast_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_fast_learn.add(BatchNormalization())

wide_model_fast_learn.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))

wide_model_fast_learn.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(MaxPooling2D(pool_size=(2, 2)))
wide_model_fast_learn.add(BatchNormalization())

wide_model_fast_learn.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
wide_model_fast_learn.add(Dropout(0.2))

wide_model_fast_learn.add(GlobalAveragePooling2D())
wide_model_fast_learn.add(Dense(num_classes, activation='softmax'))
    
# Compile model

adam_op = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
wide_model_fast_learn.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(wide_model_fast_learn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_202 (Conv2D)          (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_176 (MaxPoolin (None, 111, 111, 64)      0         
_________________________________________________________________
batch_normalization_127 (Bat (None, 111, 111, 64)      256       
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_177 (MaxPoolin (None, 54, 54, 64)        0         
_________________________________________________________________
batch_normalization_128 (Bat (None, 54, 54, 64)        256       
_________________________________________________________________
conv2d_204 (Conv2D)          (None, 52, 52, 64)        36928     
__________

In [83]:
wide_model_fast_learn.fit_generator(train_generator, validation_data=validation_generator,
                         steps_per_epoch=800, epochs=10, callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 215s 268ms/step - loss: 5.0008 - acc: 0.0029 - val_loss: 4.9147 - val_acc: 0.0083
Epoch 2/10
800/800 [==============================] - 208s 259ms/step - loss: 4.8804 - acc: 0.0038 - val_loss: 5.1029 - val_acc: 0.0063
Epoch 3/10
800/800 [==============================] - 200s 250ms/step - loss: 4.8519 - acc: 0.0000e+00 - val_loss: 4.7985 - val_acc: 0.0083
Epoch 4/10
 27/800 [>.............................] - ETA: 1:54 - loss: 4.8807 - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
ImageDataGenerator?